<a href="https://colab.research.google.com/github/Yunpei24/BigDataBase/blob/main/Activite2_Question1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ACTIVITE2





Installation du JDK

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Création de la variable d'environnement <JAVA_HOME> pour situer l'emplacement d'installationde Java 

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Installation du framework Hadoop

Téléchargement depuis les archives de la fondation Apache

In [ ]:
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz

Extraction de l'archive

In [ ]:
!tar -xzvf hadoop-3.3.0.tar.gz

Copie du dossier extrait dans l'emplacement <user/local>

In [5]:
!cp -r hadoop-3.3.0/ /usr/local/

## Programmation de tâches distribuées avec <i>MapReduce</i>

Création d'un repertoire <myinput> pour contenir le jeu de données à tester durant cet exercice e d'un second pour les résultats du traitement distribué

In [6]:
!mkdir -p ~/myinput
!mkdir -p ~/myoutput

Télachargement du jeu de données dans le fichier <u>purchases.txt</u>

In [ ]:
!curl -L -o 'purchases.txt' 'https://drive.google.com/u/0/uc?id=1NS-PSXW8bSNpzFH4XRbtmMnMGhXBdYy6&export=download&confirm=t'

Déplacement du fichier <u>purchases.txt</u> dans le répertoire <myinput>

In [8]:
!mv purchases.txt ~/myinput/purchases.txt

Vérification que les fichiers ont été bien copiés

In [9]:
!ls /usr/myinput

ls: cannot access '/usr/myinput': No such file or directory


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Affichage des premiers lignes du fichier. Le format des enregistrement est le suivant:
<table border='1'><tr>
<td>Date</td><td>Heure</td><td>Magasin</td><td>Produit</td><td>Montant</td><td>Moyen_de_paiement</td>
</tr></table>
La tabulation <b>\t</b> est utilisée comme séparateur de colonne ✅

# QUestion 1
def mapper2(file_path):
  fichier = open(file_path, 'r')
  file_line = fichier.readlines()
  fichier.close()
  result = []

  for line in file_line:
    data = line.strip().split("\t")
    if len(data) == 6:
      date, time, store, item, montant, payment = data
      result.append(payment)
  
  return result

def reducer2(file_path):
  map = mapper2(file_path)
  fic = {}
  for elm in map:
    if elm in fic:
      fic.update({elm : int(fic[elm]) + 1})
    else:
      fic.update({elm : 0})
  
  for key in fic:
    print(key, "\t", fic[key]) 

file_path = '/root/myinput/purchases.txt'  
reducer2(file_path)

**PARTIE MAPPER**

In [11]:
#!/usr/bin/python
import sys
# Mettez vos instructions ...
for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) == 6:
    date, time, store, item, amount, payment = data
    print(payment + "\t1")

In [13]:
# All cell codes are stored in a List variable "In"
with open('/content/mapper.py', 'w') as f:
  f.write(In[11]) 
f.close()

In [14]:
!chmod u+rwx /content/mapper.py

In [15]:
!head -10 ~/myinput/purchases.txt | python3 /content/mapper.py

Amex	1
Visa	1
Cash	1
Discover	1
MasterCard	1
MasterCard	1
Visa	1
Cash	1
Discover	1
Visa	1


**PARTIE REDUCER**

In [21]:
#!/usr/bin/python
from operator import itemgetter
import sys
# Mettez vos instructions ...
salesTotal = 0
oldKey = None

for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) != 2:
    continue
  
  thisKey, thisSale = data
  if oldKey and oldKey != thisKey:
    print(oldKey, "\t", str(salesTotal))
    salesTotal = 0
    
  oldKey = thisKey
  salesTotal += int (thisSale)

if oldKey != None:
  print(oldKey, "\t", str(salesTotal))

In [22]:
# All cell codes are stored in a List variable "In"
with open('/content/reducer.py', 'w') as f:
  f.write(In[16]) 
f.close()

In [23]:
!chmod u+rwx /content/reducer.py

Test du traitement distribué sur quelques enregistrements

In [24]:
!head -50 ~/myinput/purchases.txt | python3 /content/mapper.py | sort | python3 /content/reducer.py

Amex 	 10
Cash 	 10
Discover 	 8
MasterCard 	 9
Visa 	 13


In [ ]:
!rm -r ~/myoutput
!/usr/local/hadoop-3.3.0/bin/hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input ~/myinput -output ~/myoutput -file /content/mapper.py  -file /content/reducer.py  -mapper 'python mapper.py'  -reducer 'python reducer.py'

In [26]:
!ls ~/myoutput

part-00000  _SUCCESS


In [27]:
!tail -n 30 ~/myoutput/part-00000

Amex 	 826535
Cash 	 828770
Discover 	 827426
MasterCard 	 828524
Visa 	 827221
